<a href="https://colab.research.google.com/github/Kirankumarpetlu/CropYieldPrediction-MLR/blob/main/NIDSmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.5 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, LSTM, SimpleRNN
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scapy.all import sniff, IP, TCP, UDP

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

data_path = {
    "/content/Monday-WorkingHours.pcap_ISCX.csv": "Normal",
    "/content/Tuesday-WorkingHours.pcap_ISCX.csv": "Brute Force Attack",
    "/content/Wednesday-WorkingHours.pcap_ISCX.csv": "DoS and Heartbleed Attacks",
    "/content/Thursday-WorkingHours-MorningWebAttacks.pcap_ISCX.csv": "Web-based Attacks",
    "/content/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv": "Infiltration Attacks",
    "/content/Friday-WorkingHours-Morning.pcap_ISCX.csv": "",
    "/content/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv": "Port Scanning Attack",
    "/content/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv": ""
}
file_attack_mapping = {
    file_path: attack_type
    for file_path, attack_type in data_path.items()
}

def load_data():
    df_list = []
    for file, attack_type in file_attack_mapping.items():
        if os.path.exists(file):
            temp_df = pd.read_csv(file, low_memory=False)
            temp_df['Attack_Type'] = attack_type
            df_list.append(temp_df)
        else:
            print(f"Warning: File not found: {file}. Skipping...")
    df = pd.concat(df_list, ignore_index=True)
    return df

def preprocess_data(df):
    df = df.drop(columns=['Flow ID', 'Source IP', 'Destination IP', 'Timestamp'], errors='ignore')

    # Replace infinite values with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Drop rows with NaN values
    df.dropna(inplace=True)

    df['Label'] = df['Attack_Type'].apply(lambda x: 1 if x != "Normal" else 0)

    for column in df.select_dtypes(include=['object']).columns:
        if column not in ['Label', 'Attack_Type']:
            label_encoder = LabelEncoder()
            df[column] = label_encoder.fit_transform(df[column])

    scaler = StandardScaler()
    X = scaler.fit_transform(df.drop(columns=['Label', 'Attack_Type']))
    y = df['Label'].values
    return X, y

df = load_data()
X, y = preprocess_data(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
# CNN Model
from tensorflow.keras.layers import MaxPooling1D, Dropout # Import Dropout

def build_cnn():
    model = Sequential([
        Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# LSTM Model
def build_lstm():
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        LSTM(32),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# RNN Model
def build_rnn():
    model = Sequential([
        SimpleRNN(64, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        SimpleRNN(32),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def train_and_save_model(model, name):
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
    model.save(f"/content/nids_{name}.h5")
    print(f"✅ Model {name} saved!")
    return max(history.history['val_accuracy'])

cnn_model = build_cnn()
cnn_acc = train_and_save_model(cnn_model, "cnn")

lstm_model = build_lstm()
lstm_acc = train_and_save_model(lstm_model, "lstm")

rnn_model = build_rnn()
rnn_acc = train_and_save_model(rnn_model, "rnn")

# Select the Best Model
best_model_name = max(("cnn", cnn_acc), ("lstm", lstm_acc), ("rnn", rnn_acc), key=lambda x: x[1])[0]
best_model = tf.keras.models.load_model(f"/content/nids_{best_model_name}.h5")
print(f"✅ Loaded best model: {best_model_name}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 43s 4ms/step - accuracy: 0.8309 - loss: 0.4010 - val_accuracy: 0.8388 - val_loss: 0.3740
Epoch 2/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step - accuracy: 0.8370 - loss: 0.3798 - val_accuracy: 0.8417 - val_loss: 0.3680
Epoch 3/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step - accuracy: 0.8422 - loss: 0.3703 - val_accuracy: 0.8473 - val_loss: 0.3617
Epoch 4/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 33s 3ms/step - accuracy: 0.8417 - loss: 0.3688 - val_accuracy: 0.8490 - val_loss: 0.3569
Epoch 5/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.8423 - loss: 0.3660 - val_accuracy: 0.8496 - val_loss: 0.3548
Epoch 6/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.8446 - loss: 0.3616 - val_accuracy: 0.8506 - val_loss: 0.3547
Epoch 7/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.8444 - loss: 0.3625 - val_accuracy: 0.8510 - val_loss: 0.3545
Epoch 8/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 

✅ Model cnn saved!


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 134s 13ms/step - accuracy: 0.8297 - loss: 0.4168 - val_accuracy: 0.8339 - val_loss: 0.3816
Epoch 2/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 132s 12ms/step - accuracy: 0.8353 - loss: 0.3795 - val_accuracy: 0.8433 - val_loss: 0.3700
Epoch 3/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 151s 13ms/step - accuracy: 0.8414 - loss: 0.3699 - val_accuracy: 0.8463 - val_loss: 0.3620
Epoch 4/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 131s 13ms/step - accuracy: 0.8440 - loss: 0.3652 - val_accuracy: 0.8416 - val_loss: 0.3646
Epoch 5/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 143s 13ms/step - accuracy: 0.8442 - loss: 0.3617 - val_accuracy: 0.8492 - val_loss: 0.3542
Epoch 6/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 134s 12ms/step - accuracy: 0.8476 - loss: 0.3567 - val_accuracy: 0.8495 - val_loss: 0.3537
Epoch 7/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 133s 13ms/step - accuracy: 0.8474 - loss: 0.3555 - val_accuracy: 0.8502 - val_loss: 0.3515
Epoch 8/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 140s 13ms/s

✅ Model lstm saved!
Epoch 1/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 149s 14ms/step - accuracy: 0.8290 - loss: 0.4258 - val_accuracy: 0.8342 - val_loss: 0.4236
Epoch 2/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 188s 13ms/step - accuracy: 0.8317 - loss: 0.4205 - val_accuracy: 0.8313 - val_loss: 0.4250
Epoch 3/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 140s 13ms/step - accuracy: 0.8309 - loss: 0.4279 - val_accuracy: 0.8351 - val_loss: 0.4257
Epoch 4/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 143s 13ms/step - accuracy: 0.8316 - loss: 0.4250 - val_accuracy: 0.8341 - val_loss: 0.4251
Epoch 5/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 152s 14ms/step - accuracy: 0.8328 - loss: 0.4275 - val_accuracy: 0.8348 - val_loss: 0.4231
Epoch 6/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 133s 13ms/step - accuracy: 0.8337 - loss: 0.4255 - val_accuracy: 0.8329 - val_loss: 0.4267
Epoch 7/10
10332/10332 ━━━━━━━━━━━━━━━━━━━━ 131s 13ms/step - accuracy: 0.8323 - loss: 0.4231 - val_accuracy: 0.8310 - val_loss: 0.4287
Epoch 8/10
10332/10332 ━━━━━━━━━━━━

✅ Model rnn saved!
✅ Loaded best model: lstm


In [ ]:

def live_packet_sniffing(model):
    def process_packet(packet):
        features = []
        if packet.haslayer(IP):
            features.append(packet[IP].len)
            features.append(packet[IP].ttl)
            features.append(packet[IP].proto)
        else:
            features.extend([0, 0, 0])
        if packet.haslayer(TCP) or packet.haslayer(UDP):
            features.append(packet.sport)
            features.append(packet.dport)
        else:
            features.extend([0, 0])
        features = np.array(features).reshape(1, -1)
        prediction = model.predict(features)
        attack_detected = prediction[0][0] > 0.5
        status = "Intrusion" if attack_detected else "Normal"
        socketio.emit('update', {'status': status})
        print(f"Packet: {features} -> {status}")
    sniff(prn=process_packet, store=False)
